# LSTM Models

Created: 2019.10.18  
Notebook sequence: 8

For experimenting with keras lstm models

---

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import util
import pandas as pd
import keras
import numpy as np
from tqdm import tqdm_notebook as tqdm
from keras.layers import LSTM, TimeDistributed, Dense, Dropout, Activation, Masking
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [4]:
#data = util.load_dataset("os_reliable_w2v/sequence.pkl")
#df = util.load_selection_dataset("os_reliability.csv")

data = util.load_dataset("as_biased_w2v/sequence.pkl")
df = util.load_selection_dataset("as_bias.csv")

In [11]:
y = list(df.biased)
X = data
X_train, X_valid, Y_train, Y_valid = train_test_split(X, y, test_size=0.20, random_state=12)
# X_train = X
# Y_train = y
print(np.array(X_train).shape)
print(X_train[0].shape)
print(X_train[1].shape)

maximum = 0
lengths = []
for entry in tqdm(X_train):
    lengths.append(entry.shape[0])
lengths = np.array(lengths)

(11916,)
(541, 300)
(190, 300)


In [27]:
max(lengths)

32683

In [28]:
lengths.mean()

525.4841221886539

In [6]:
model = keras.Sequential()
#model.add(LSTM(32, return_sequences=True, input_shape=(None, 300)))
#model.add(TimeDistributed(Dense(2, activation='sigmoid')))
#model.add(LSTM(64, return_sequences=True, input_shape=(100, 300)))
model.add(Masking(mask_value=0.0, input_shape=(500, 300)))
# dropout=0.0, recurrent_dropout=0.0
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
#model.add(LSTM(64))
#model.add(LSTM(256, return_sequences=True))
#model.add(LSTM(128))
#model.add(Dropout(.2))
#model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#model.add(Activation("softmax"))

In [12]:

X_train = np.array(X_train)
Y_train = np.array(Y_train)
#Y_train = keras.utils.to_categorical(Y_train)
#Y_train = np.expand_dims(Y_train, axis=1)
# Y_train = np.expand_dims(Y_train, axis=1)
X_train_padded = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=500, padding='post', dtype='float32')


In [8]:
X_train.shape

(11916,)

In [9]:
X_train_padded.shape

(11916, 500, 300)

In [24]:
X_train_padded[0]

array([[-0.10595703,  0.21386719,  0.11865234, ...,  0.10693359,
         0.02368164, -0.03540039],
       [-0.02233887, -0.00454712, -0.10009766, ..., -0.1796875 ,
        -0.12158203, -0.09082031],
       [-0.30859375,  0.18945312, -0.2734375 , ..., -0.05517578,
         0.2265625 ,  0.24121094],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [9]:
print(Y_train.shape)
print(X_train_padded.shape)


(14895, 2)
(14895, 500, 300)


In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()
model.fit(X_train_padded, Y_train, batch_size=128, validation_split=.2, epochs=100, verbose=1, use_multiprocessing=True, workers=4)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_1 (Masking)          (None, 500, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                42624     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 42,657
Trainable params: 42,657
Non-trainable params: 0
_________________________________________________________________
Train on 9532 samples, validate on 2384 samples
Epoch 1/100
9532/9532 [==============================] - 47s 5ms/step - loss: 0.6255 - binary_accuracy: 0.6667 - val_loss: 0.5981 - val_binary_accuracy: 0.6921
Epoch 2/100
9532/9532 [==============================] - 45s 5ms/step - loss: 0.5957 - binary_accuracy: 0.6782 - val_loss: 0.5816 - val_binary_accuracy: 0.7005
Epoch 3/100


## LSTM on avg

In [17]:
data = util.load_dataset("os_reliable_w2v/avg.pkl")
df = util.load_selection_dataset("os_reliability.csv")

y = list(df.reliable)
X = np.array(data)
X = np.expand_dims(X, axis=1)
#X_train, X_valid, Y_train, Y_valid = train_test_split(X, y, test_size=0.20, random_state=13)

In [18]:
X.shape

(9352, 1, 300)

In [26]:
model = keras.Sequential()
#model.add(LSTM(32, return_sequences=True, input_shape=(None, 300)))
#model.add(TimeDistributed(Dense(2, activation='sigmoid')))
model.add(LSTM(128, return_sequences=True, input_shape=(1, 300)))
model.add(LSTM(128))
model.add(Dropout(.8))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [28]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X, y, batch_size=64, validation_split=.2, epochs=100, verbose=1, use_multiprocessing=True, workers=4)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 1, 128)            219648    
_________________________________________________________________
lstm_15 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 129       
Total params: 367,873
Trainable params: 367,873
Non-trainable params: 0
_________________________________________________________________
Train on 7481 samples, validate on 1871 samples
Epoch 1/100
7481/7481 [==============================] - 2s 288us/